In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('Movie Dataset 2023.csv')

In [ ]:
movies.isnull().sum()

In [ ]:
movies = movies.dropna()

In [ ]:
movies.isnull().sum()

In [ ]:
movies = movies[['movie_id', 'movie_name', 'year', 'overview', 'genre', 'director', 'cast']]

In [ ]:
movies = movies.rename(columns = {'movie_name':'title'})

In [ ]:
movie = movies.copy()

In [ ]:
movie = movie[~(movie == " ").any(axis=1)]

In [ ]:
movie

In [ ]:
movie['genre'] = movie['genre'].apply(lambda x: [genre.strip() for genre in x.split(',')])

In [ ]:
movie['director'] = movie['director'].apply(lambda x: [director.strip() for director in x.split(',')])

In [ ]:
movie['cast'] = movie['cast'].apply(lambda x: [cast.strip() for cast in x.split(',')])

In [ ]:
movie['overview'] = movie['overview'].apply(lambda x:x.split())

In [ ]:
movie['genre'] = movie['genre'].apply(lambda x: [i.replace(" ","") for i in x])
movie['director'] = movie['director'].apply(lambda x: [i.replace(" ","") for i in x])
movie['cast'] = movie['cast'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movie.head()

In [ ]:
movie['tags'] = movie['overview'] + movie['genre'] + movie['director'] + movie['cast']

In [ ]:
movie.head()

In [ ]:
df = movie[['movie_id', 'title', 'year', 'tags']]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df['tags'] = df['tags'].apply(lambda x:" ".join(x)) 

In [ ]:
df['tags'] = df['tags'].apply(lambda x:x.lower()) 

In [ ]:
df['tags'][0]

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y= []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [ ]:
df['tags'] = df['tags'].apply(stem)

In [ ]:
ps.stem('successfully')

In [ ]:
df['tags'][1]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=4000, stop_words= 'english')

In [ ]:
vectors = cv.fit_transform(df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[1])), reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
import requests

In [ ]:
def recommend(moviez):
    movie_index = df[df['title'] == moviez].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:9]
    
    for i in movies_list:
        print(df.iloc[i[0]])
recommend('Ghoomer')

In [ ]:
def recommend(moviez):
    movie_index = df[df['title'] == moviez].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:9]
    
    recommended_movies = []
    recommended_movies_posters = []
    for i in movies_list:
        movie_id = df.iloc[i[0]].title
        recommended_movies.append(df.iloc[i[0]].title)
        recommended_movies_posters.append(fetch_poster(movie_id, df.iloc[i[0]].year))
        
def fetch_poster(movie_id, year):
    response = requests.get('https://www.omdbapi.com/?t={}&y={}&apikey=3ac1a5bd'.format(movie_id, year))
    data = response.json()
    print(data)
    
recommend('Jaane Jaan')

In [ ]:
import pickle 

In [ ]:
pickle.dump(df.to_dict(), open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity, open('similarity.pkl','wb'))